In [16]:
import sys
print(sys.version)
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install iexfinance
!{sys.executable} -m pip install --no-deps empyrical
!{sys.executable} -m pip install --no-deps pandas-datareader
!{sys.executable} -m pip show scipy
!{sys.executable} -m pip install pyfolio
from datetime import datetime
from iexfinance.stocks import get_historical_data
from iexfinance.stocks import Stock
import numpy as numpy
import pandas as pandas
try:
    import google.colab
    IN_COLAB = True
    print("The package pyfolio works in Google Colab, but not in WinPython")
except:
    IN_COLAB = False
if (IN_COLAB):
    import pyfolio as pyfolio
else:
    print("pyfolio not installed")

3.7.6 (tags/v3.7.6:43364a7ae0, Dec 19 2019, 00:42:30) [MSC v.1916 64 bit (AMD64)]
Requirement already up-to-date: pip in d:\program\winpython64-3760\python-3.7.6.amd64\lib\site-packages (20.0.2)
Name: scipy
Version: 1.4.1
Summary: SciPy: Scientific Library for Python
Home-page: https://www.scipy.org
Author: None
Author-email: None
License: BSD
Location: d:\program\winpython64-3760\python-3.7.6.amd64\lib\site-packages
Requires: numpy
Required-by: statsmodels, seaborn, scs, scikit-optimize, scikit-learn, scikit-fuzzy, QuantStats, quantecon, pygbm, pyfolio, pyflux, plotnine, pdvega, osqp, oct2py, mlxtend, lmfit, Keras, imbalanced-learn, guiqwt, empyrical, ecos, datashader, dask-searchcv, cvxpy, astroML
pyfolio not installed


In [17]:
# get iextoken
import json
import os
print(os.getcwd())
def GetIexToken():
    strRoot='D:\\denbrige\\180 FxOption\\103 FxOptionVerBack\\083 FX-Git-Pull\\14docktrader\\config\\'
    strToken=''
    try:
        with open(strRoot + 'iex.conf') as fJsn:
            jsnIEX = json.load(fJsn)
        strToken=jsnIEX['iextoken']
        print("Found IEX Token from config")
    except:
        strToken=input("Enter IEX Token: ")
    return strToken
strToken=GetIexToken()

D:\Program\WinPython64-3760\notebooks
Found IEX Token from config


In [18]:
# list of tickers to be processed
def GetStockTicker(blnSp500=False):
    strTicker = []
    if blnSp500 == True:
        strStockFile = "stockrow/sp500_constituents.csv"
        if os.path.exists(strStockFile) == True:
            file=open(strStockFile, "r")
            reader = csv.reader(file)
            #skip header
            next(reader) 
            for line in reader:
                strTicker.append(line[0])
        else:
            print("Error: " + strStockFile + " undefined")
    else:
        strInput = input("Enter ticker(s) delimited by comma: ")
        for ticker in strInput.split(','):
            strTicker.append(ticker)
    return strTicker

def IexStockAsset(ticker):
    # assert ticker is not empty
    if len(ticker) == 0:
        return
    
    start=datetime(2019, 2, 9)
    end=datetime(2020,2,11)
    
    dfRet = pandas.DataFrame(columns=ticker)
    
    for i in ticker:
        ohlc = get_historical_data(i, start, end, output_format='pandas', token=strToken)
        ohlc.columns=["Open", "High", "Low", i, "Volume"]
        dfRet[i] = ohlc[i]
        
    return dfRet

In [19]:
strTicker = GetStockTicker()
print('No. of tickers:', len(strTicker))
dfRisky = IexStockAsset(strTicker)
dfReturns = dfRisky.pct_change().dropna()
print(dfReturns)
intRisky = len(strTicker)
print(intRisky)
dblWeights = intRisky * [1 / intRisky]
PortfolioReturns = pandas.Series(numpy.dot(dblWeights, dfReturns.T), index=dfReturns.index)
print(dfReturns.T)
print(PortfolioReturns)
pyfolio.create_simple_tear_sheet(PortfolioReturns)

Enter ticker(s) delimited by comma: FB
No. of tickers: 1
                  FB
date                
2019-02-12 -0.004524
2019-02-13 -0.005877
2019-02-14 -0.000731
2019-02-15 -0.008844
2019-02-19 -0.001292
...              ...
2020-02-05  0.001334
2020-02-06  0.003522
2020-02-07  0.007019
2020-02-10  0.003438
2020-02-11 -0.027551

[252 rows x 1 columns]
1
date  2019-02-12  2019-02-13  2019-02-14  2019-02-15  2019-02-19  2019-02-20  \
FB     -0.004524   -0.005877   -0.000731   -0.008844   -0.001292    0.001664   

date  2019-02-21  2019-02-22  2019-02-25  2019-02-26  ...  2020-01-29  \
FB     -0.015502     0.01156    0.016863   -0.002977  ...    0.024978   

date  2020-01-30  2020-01-31  2020-02-03  2020-02-04  2020-02-05  2020-02-06  \
FB     -0.061372   -0.036367    0.011292    0.027621    0.001334    0.003522   

date  2020-02-07  2020-02-10  2020-02-11  
FB      0.007019    0.003438   -0.027551  

[1 rows x 252 columns]
date
2019-02-12   -0.004524
2019-02-13   -0.005877
2019-02-14   -

NameError: name 'pyfolio' is not defined